In [ ]:
# default_exp data_import

In [ ]:
#hide
import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)

In [ ]:
#export
from criteo_nbdev.constants import *

In [ ]:
#hide
!gcloud config set project $PROJECT_ID

In [ ]:
#export
from google.cloud import bigquery
from google.api_core.exceptions import GoogleAPIError
from criteo_nbdev.constants import *

def create_bigquery_dataset_if_necessary(dataset_id):
    # Construct a full Dataset object to send to the API.
    client = bigquery.Client(project=PROJECT_ID)
    dataset = bigquery.Dataset(
        bigquery.dataset.DatasetReference(PROJECT_ID, dataset_id))
    dataset.location = LOCATION

    try:
        dataset = client.create_dataset(dataset)  # API request
        return True
    except GoogleAPIError as err:
        if err.code != 409:  # http_client.CONFLICT
            raise
    return False

In [ ]:
#export
def load_data_into_bigquery(source_uris, dataset_id, table_id):
    create_bigquery_dataset_if_necessary(dataset_id)
    client = bigquery.Client(project=PROJECT_ID)
    dataset_ref = client.dataset(dataset_id)
    table_ref = dataset_ref.table(table_id)
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.source_format = bigquery.SourceFormat.CSV
    job_config.field_delimiter = '\t'
    job_config.schema = CSV_SCHEMA

    load_job = client.load_table_from_uri(
        source_uris, table_ref, job_config=job_config
    )
    print("Starting job {}".format(load_job.job_id))

    try:
        load_job.result()  # Waits for table load to complete.
        print("Job finished.")
    except Exception as err:
        print(load_job.errors)
        print(err)

    destination_table = client.get_table(table_ref)
    print("Loaded {} rows.".format(destination_table.num_rows))

In [ ]:
#hide
from criteo_nbdev.constants import *
from criteo_nbdev.core import skip

skip()

print(URL)
print(DATASET_ID)
print(TABLE_ID)

load_data_into_bigquery(URL, DATASET_ID, TABLE_ID)

In [ ]:
#hide
%load_ext google.cloud.bigquery

In [ ]:
#hide
%%bigquery --project alekseyv-scalableai-dev --verbose
SELECT
count(*)
FROM `alekseyv-scalableai-dev.criteo_kaggle_2.days`

In [ ]:
#export

import re
from tensorflow.python.platform import gfile
from criteo_nbdev.constants import *

def _get_file_names(file_pattern):
    if isinstance(file_pattern, list):
        if not file_pattern:
            raise ValueError("File pattern is empty.")
        file_names = []
        for entry in file_pattern:
            file_names.extend(gfile.Glob(entry))
    else:
        file_names = list(gfile.Glob(file_pattern))
    return file_names
        
def _get_file_suffix(file_name):
    return int(re.search(GCS_FOLDER + 'train_(\d+)', file_name).group(1))

def get_file_names_with_validation_split(dataset_size: DATASET_SIZE_TYPE, dataset_type:DATASET_TYPE, validation_split):
    file_names = _get_file_names(GCS_FOLDER + 'train_0*')
    modulo = int(len(file_names) * validation_split)
    large_dataset_file_names = list(
        file_name for file_name in file_names \
        if (dataset_type == DATASET_TYPE.training) ^ (_get_file_suffix(file_name) % modulo == 0))
    if dataset_size == DATASET_SIZE_TYPE.full:
        return large_dataset_file_names
    
    modulo = None
    if dataset_size == DATASET_SIZE_TYPE.small:
        modulo = 10
    elif dataset_size == DATASET_SIZE_TYPE.tiny:
        modulo = 100
    else:
        raise ValueError('Invalid dataset_size:%s', dataset_size)
    small_dataset_files = list(
        file_name for (idx, file_name) in enumerate(large_dataset_file_names) if idx % modulo == 0)
    return small_dataset_files


In [ ]:
#hide
import gcp_runner.core
gcp_runner.core.export_and_reload_all(silent=True)

get_file_names_with_validation_split(DATASET_SIZE_TYPE.tiny, DATASET_TYPE.validation, 0.2)

In [ ]:
#hide
from criteo_nbdev.core import skip; skip()

file_names = get_file_names_with_validation_split(DATASET_SIZE_TYPE.small, DATASET_TYPE.training, 0.2)
print(file_names)
load_data_into_bigquery(file_names, DATASET_ID, 'training_small')

file_names = get_file_names_with_validation_split(DATASET_SIZE_TYPE.small, DATASET_TYPE.validation, 0.2)
print(file_names)
load_data_into_bigquery(file_names, DATASET_ID, 'validation_small')

file_names = get_file_names_with_validation_split(DATASET_SIZE_TYPE.full, DATASET_TYPE.validation, 0.2)
print(file_names)
load_data_into_bigquery(file_names, DATASET_ID, 'validation_full')

file_names = get_file_names_with_validation_split(DATASET_SIZE_TYPE.full, DATASET_TYPE.training, 0.2)
print(file_names)
load_data_into_bigquery(file_names, DATASET_ID, 'training_full')

file_names = get_file_names_with_validation_split(DATASET_SIZE_TYPE.tiny, DATASET_TYPE.validation, 0.2)
print(file_names)
load_data_into_bigquery(file_names, DATASET_ID, 'validation_tiny')

file_names = get_file_names_with_validation_split(DATASET_SIZE_TYPE.tiny, DATASET_TYPE.training, 0.2)
print(file_names)
load_data_into_bigquery(file_names, DATASET_ID, 'training_tiny')